In [ ]:
## -----------------------------------------------------------------------------------------------------------------##
## Written by Eli Melaas, adapted for Python by Seamore Zhu 
## -----------------------------------------------------------------------------------------------------------------##

In [10]:
from joblib import Parallel, delayed
import sys
import os
import glob
import shapely
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
import rasterio
from osgeo import gdal

In [2]:
# Register parallel processing with joblib
Parallel(n_jobs=16)

Parallel(n_jobs=16)

In [3]:
# Topographic correction function

In [14]:
# Vegetation index calculation function
def calcVI(whatIndex, blue=None, green=None, red=None, nir=None, swir1=None, swir2=None):
    if (whatIndex == 'evi'): index = 2.5*(nir - red) / (nir + 2.4*red - 7.5*blue)
    if (whatIndex == 'evi2'): index = 2.5*(nir - red) / (nir + 2.4*red + 1)
    if (whatIndex == 'ndvi'): index = (nir - red) / (nir + red)
    if (whatIndex == 'ndmi'): index = (nir - swirOne) / (swirOne + nir)
    if (whatIndex == 'nbr'): index = (nir - swirTwo) / (swirTwo + nir)
    if (whatIndex == 'nbr2'): index = (swirOne - swirTwo) / (swirOne + swirTwo)
    if (whatIndex == 'savi'): index = ((nir - red) / (nir + red + 0.5)) * 1.5
    if (whatIndex == 'msavi'): index = (2*nir + 1 - sqrt((2*nir + 1)^2 - 8*(nir - red))) / 2
    if (whatIndex == 'rcc'): index = red / (red + green + blue)
    if (whatIndex == 'gcc'): index = green / (red + green + blue)
    if (whatIndex == 'grvi'): index = (green-red) / (red + green)
    if (whatIndex == 'ndsi'): index = (green-swirOne) / (green+swirOne)
    if (whatIndex == 'npci'): index = (red - blue) / (red + blue)  #Hatfield et al. 2010
    if (whatIndex == 'CIgreen'): index = (nir/green) - 1           #Hatfield et al. 2010
    if (whatIndex == 'psri'): index = (red-green) / (nir)          #Hatfield et al. 2010
    return index

In [6]:
#args = sys.argv[1:]  # Get command-line arguments
#tile = args[0]       # Extract the first argument to tile name
#data_dir = args[1]   # Extract the second argument to data directory
#tile_dir = args[2]   # Extract the third argument to tile directory
#index = args[3]      # Extract the fourth argument to vegetation index

tile = 'h15v02'  # Will be parameterized in json file and passed as argument using shell script (see above code)
data_dir = '/projectnb/modislc/users/seamorez/Landsat_Pheno/ARD/'  # Will be parameterized in json file
tile_dir = '/projectnb/modislc/projects/landsat_sentinel/ARD/'  # Will be parameterized in json file
index = 'evi2' # Will be parameterized in json file

H = int(tile[1:3])
V = int(tile[4:6])

os.chdir(data_dir+tile) 

In [10]:
# Generate ARD tile and save to directory (NOTE: THIS IS FOR AK, NOT CONUS)
ard_tiles = gpd.read_file(tile_dir+'Alaska_C2_ARD_grid/ak_c2_ard_grid.shp')
tile_shp = ard_tiles.loc[(ard_tiles['h']==H) & (ard_tiles['v']==V)]

# Determine lat/lon extent of tile for NED download (NOTE: THIS MAY NEED EDITING WHEN DOING NEW TOPO CORRECTION FUNCTION)
tile_proj = tile_shp.to_crs("+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=23 +lon_0=-96 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0")
tile_proj.to_file(os.getcwd()+'/SHP/'+'tile'+'.shp')

      h  v                                           geometry
301  15  2  POLYGON ((48285.000 2024325.000, 48285.000 217...


In [ ]:
# Generate DEM, Slope, Aspect, and LC Maps

In [7]:
# Load in SR Bands, QA Layer, SZA, and SAA

os.chdir(data_dir+tile+'/IMG')
in_dirs = glob.glob("L*", recursive=True)

In [8]:
# Check for images with missing evi outputs
os.chdir(data_dir+tile)
in_dirs_tile = glob.glob("**/evi2.tif", recursive=True)  # NOTE: ADD '_topocorr' IF USING TOPO CORRECTION
in_dirs_tile = [os.path.abspath(dir_tile) for dir_tile in in_dirs_tile]
ydoy = [os.path.basename(dir_SR)[15:23] for dir_SR in in_dirs_tile]
ydoy2 = [os.path.basename(dir_SR)[15:23] for dir_SR in in_dirs]
w = [i for i, d in enumerate(ydoy2) if d not in ydoy]

In [35]:
os.chdir(data_dir+tile+'/IMG')

# Extract metadata for easier raster writing
meta_file = rasterio.open(os.path.join(os.getcwd(), in_dirs[0], in_dirs[0] + '_02_SR_B4.TIF'),'r')
meta = meta_file.meta
meta.update(dtype='int16',nodata=32767)

# Output vegetation index images
for i in w:
    # Landsat 8-9
    if (in_dirs[i][3] == '8') | (in_dirs[i][3] == '9'):
        nir = gdal.Open(os.path.join(os.getcwd(), in_dirs[i], in_dirs[i] + '_02_SR_B5.TIF'))
        red = gdal.Open(os.path.join(in_dirs[i], in_dirs[i] + '_02_SR_B4.TIF'))
        QA = gdal.Open(os.path.join(in_dirs[i], in_dirs[i] + '_02_QA_PIXEL.TIF'))

        s = [red, nir, QA]
        band_vals = [band.ReadAsArray() for band in s]
        band_vals = np.array(band_vals).astype(float)
        band_vals[band_vals<=0] = np.nan

        qa = band_vals[2,]
        k = np.where((qa!=21824)&(qa!=22080)&(qa!=54596)&(qa!=54852))  # L8-9 C2 ARD QA_PIXEL values for clear imagery
        #w <- which(qa!=322 & qa!=386 & qa!=834 & qa!=898 & qa!=1346)
        band_vals[0:2,k[0],k[1]] = np.nan
        
    # Landsat 4-7
    else:
        nir = gdal.Open(os.path.join(os.getcwd(), in_dirs[i], in_dirs[i] + '_02_SR_B4.TIF'))
        red = gdal.Open(os.path.join(os.getcwd(), in_dirs[i], in_dirs[i] + '_02_SR_B3.TIF'))
        QA = gdal.Open(os.path.join(os.getcwd(), in_dirs[i], in_dirs[i] + '_02_QA_PIXEL.TIF'))

        s = [red, nir, QA]
        band_vals = [band.ReadAsArray() for band in s]
        band_vals = np.array(band_vals).astype(float)
        band_vals[band_vals<=0] = np.nan
        
        qa = band_vals[2,]
        k = np.where((qa!=5440)&(qa!=5696))  # L4-7 C2 ARD QA_PIXEL values for clear imagery
        #w <- which(qa!=66 & qa!=130)
        band_vals[0:2,k[0],k[1]] = np.nan

    vi = calcVI(red=band_vals[0,],nir=band_vals[1,],whatIndex=index)    
    vi = (vi*10000).round()
    vi[(vi < 0) | (np.isinf(vi)) | (np.isnan(vi))] = 32767  # Remove negative, infinite, and nan values
    vi = vi.astype('int16')
    with rasterio.open(in_dirs[i]+'/evi2.tif', 'w', **meta) as dst:
        dst.write(vi, 1)

[[32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 ...
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]]
[[32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 ...
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]]
[[ 4081  4319  4433 ...  4452  4223  4106]
 [ 4468  4835  4775 ...  4340  4154  4225]
 [ 4918  5040  5077 ...  4004  4015  4064]
 ...
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]]
[[32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 [32767 32767 32767 ... 32767 32767 32767]
 ...
 [32767 32767 32767 ...     0     0     0]
 [32767 32767 32767 ...     0  

KeyboardInterrupt: 

In [22]:
# Topographic correction output of vegetation index images

dtype('uint16')